# Research Project Experiment

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def fitness(x):
    # Add fitness function here
    return np.sum(x)

def uniform_crossover(p_c, p1, p2):
    if (np.random.random() < p_c):
        c1 = np.zeros(p1.shape)
        c2 = np.zeros(p2.shape)
        ind_l, = p1.shape
        for i in range(ind_l):
            if (np.random.random() < 0.5):
                c1[i] = p1[i]
                c2[i] = p2[i]
            else:
                c1[i] = p2[i]
                c2[i] = p1[i] 
    else:
        c1 = p1.copy()
        c2 = p2.copy()
    # Return children
    return c1, c2

def mutation(p_m, p):
    c = p.copy()
    ind_l, = c.shape
    # Return child
    for i in range(ind_l):
        if (np.random.random() < p_m):
            if (c[i] == 1):
                c[i] = 0
            else:
                c[i] = 1
    return c

def tournament_selection(fitnesses, ts):
    # Return index of selected individual
    tourn = np.random.choice(len(fitnesses), ts, replace=False)
    tourn_fits = [fitnesses[i] for i in tourn]
    return tourn[np.argmax(tourn_fits)]

pop_size = 100
ind_len = 50
p_c = 0.5
p_m = 0.01

# Initialize binary population
population = np.random.randint(0, 2, (pop_size,ind_len))

# Start at generation 0
gen = 0

# Track relevant fitness info
mean_fits = []
max_fits = []

while (gen < 100):
    # Initialize children
    children = np.zeros((pop_size, ind_len))
    # Calculate fitnesses
    fitnesses = [fitness(population[i]) for i in range(pop_size)]
    
    # Track relevant fitness info
    mean_fits.append(np.mean(fitnesses))
    max_fits.append(np.max(fitnesses))
    
    # Print generation info and increment gen
    print("Gen", gen, "Fit", max_fits[-1])
    
    # Select parents
    parents = [tournament_selection(fitnesses,2) for i in range(pop_size)]
    
    # Create children in pairs
    for i in range(0,pop_size,2):
        c1, c2 = uniform_crossover(p_c, population[parents[i]], population[parents[i+1]])
        c1 = mutation(p_m, c1)
        c2 = mutation(p_m, c2)
        children[i,:] = c1[:]
        children[i+1,:] = c2[:]
        

    # Replace population with children
    population = children
    gen+=1